In [4]:
import re
import math
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchtext import datasets, data
from sklearn.model_selection import train_test_split
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

BATCH_SIZE = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [7]:
from string import punctuation

def preprocessing(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens if token not in punctuation and token.isalpha()]
    return ' '.join(tokens)

In [8]:
# dq = open('train.txt').readlines()

# with open('questions_wo_punct.txt', mode='w') as q:
#     with open('answers_wo_punct.txt', mode='w') as answ:
#         for line in tqdm(dq):
#             line = line.strip('\n')
#             question, answer = line.split('\t')
#             question = preprocessing(question)
#             answer = preprocessing(answer)
#             q.write(question + '\n')
#             answ.write(answer + '\n')


# with open('questions.txt', mode='w') as q:
#     with open('answers.txt', mode='w') as answ:
#         for line in tqdm(dq):
#             line = line.strip('\n')
#             question, answer = line.split('\t')
#             q.write(question + '\n')
#             answ.write(answer + '\n')

100%|██████████| 5880497/5880497 [00:35<00:00, 166816.18it/s]


In [9]:
# tokenizer = Tokenizer(BPE())
# tokenizer.pre_tokenizer = Whitespace()
# trainer = BpeTrainer(special_tokens=["[UNK]", "[EOS]", "[BOS]", "[PAD]"])
# tokenizer.train(files=["questions_wo_punct.txt", "answers_wo_punct.txt"], trainer=trainer)

In [10]:
# tokenizer.save('./tokenisers/tokenizer_wo_punct')

In [57]:
tokenizer = Tokenizer.from_file('tokenisers/tokenizer')

In [58]:
TEXT = data.Field(
    fix_length=50,
    init_token='[BOS]',
    eos_token='[EOS]',
    pad_token="[PAD]",
    unk_token="[UNK]",
    lower=True,
    tokenize=lambda x: tokenizer.encode(x).tokens,
    batch_first=True,
)

fields = (('src', TEXT), ('tgt', TEXT))

In [59]:
with open('questions.txt') as f:
    question_snt = list(map(str.strip, f.readlines()))
    
with open('answers.txt') as f:
    answer_snt = list(map(str.strip, f.readlines()))
    
examples = [data.Example.fromlist(x, fields) for x in tqdm(zip(question_snt, answer_snt),total=len(answer_snt))]
test = data.Dataset(examples[-10000:], fields)
train, valid = data.Dataset(examples[:-10000], fields).split(0.9)

100%|██████████| 5880497/5880497 [06:59<00:00, 14022.85it/s]


In [60]:
len(question_snt)==len(answer_snt)

True

In [61]:
TEXT.build_vocab(train)
TEXT.vocab.stoi=tokenizer.get_vocab()
TEXT.vocab.itos = {value: key for key, value in tokenizer.get_vocab().items()}

In [15]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [16]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=tokenizer.token_to_id('[PAD]')):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data).detach()
        return tgt_mask

In [17]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        nn.Linear(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [18]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator # Linear layer
        
    def forward(self, batch):
        src, trg = batch.src, batch.trg
        src_mask, trg_mask = batch.src_mask, batch.trg_mask
        "Take in and process masked src and target sequences."
        #return self.decode(self.encode(src, src_mask), src_mask,
        #                    tgt, tgt_mask)
        return self.decode(trg, trg_mask, self.encode(src, src_mask), src_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, tgt, tgt_mask, memory, src_mask):
    #def decode(self, memory, src_mask, tgt, tgt_mask):
        #return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
        x = self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
        x = self.generator(x)
        return x

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [19]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [20]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [21]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [22]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [23]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [24]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [25]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * np.sqrt(self.d_model)

In [26]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].clone().detach()
        return self.dropout(x)

In [27]:
class BucketIteratorWrapper(DataLoader):
    __initialized = False

    def __init__(self, iterator: data.Iterator):
#         super(BucketIteratorWrapper,self).__init__()
        self.batch_size = iterator.batch_size
        self.num_workers = 8
        self.collate_fn = None
        self.pin_memory = False
        self.drop_last = False
        self.timeout = 0
        self.worker_init_fn = None
        self.sampler = iterator
        self.batch_sampler = iterator
        self.__initialized = True

    def __iter__(self):
        return map(
            lambda batch: Batch(batch.src, batch.tgt, pad=TEXT.vocab.stoi['[PAD]']),
            self.batch_sampler.__iter__()
        )

    def __len__(self):
        return len(self.batch_sampler)
    
class MyCriterion(nn.Module):
    def __init__(self, pad_idx):
        super(MyCriterion, self).__init__()
        self.pad_idx = pad_idx
        self.criterion = nn.CrossEntropyLoss(reduction='sum', ignore_index=pad_idx)
        
    def forward(self, x, target):
        x = x.contiguous().permute(0,2,1)
        ntokens = (target != self.pad_idx).data.sum()
        
        return self.criterion(x, target) / ntokens

In [28]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [62]:
torch.cuda.empty_cache()

batch_size = BATCH_SIZE
num_epochs = 10

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train, valid, test), 
                                              batch_sizes=(batch_size, batch_size, batch_size), 
                                  sort_key=lambda x: len(x.src),
                                  shuffle=True,
                                  device=DEVICE,
                                  sort_within_batch=False)
                                  
train_iter = BucketIteratorWrapper(train_iter)
valid_iter = BucketIteratorWrapper(valid_iter)
test_iter = BucketIteratorWrapper(test_iter)

model = make_model(len(TEXT.vocab), len(TEXT.vocab), N=1)
model = model.to(DEVICE)
criterion = MyCriterion(pad_idx=TEXT.vocab.stoi['[PAD]'])
criterion.cuda()

optimizer = NoamOpt(model.src_embed[0].d_model, 1, 4000, torch.optim.Adam(model.parameters(),  lr=0, betas=(0.9, 0.98), eps=1e-9))
# share weights
#https://github.com/pytorch/examples/blob/master/word_language_model/model.py#L28
model.tgt_embed[0].lut.weight = model.generator.weight

In [30]:
def train_epoch(data_iter, model, criterion):
    total_loss = 0
    data_iter = tqdm(data_iter, total=len(data_iter))
    counter = 0
    for batch in data_iter:
        out = model.forward(batch)
        loss = criterion(out, batch.trg_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()
        #optimizer.zero_grad()
        
        total_loss += loss
        data_iter.set_postfix(loss = loss.item())
        counter +=1
        
    total_loss /= counter
    return total_loss

def valid_epoch(data_iter, model, criterion):
    total_loss = 0
    data_iter = tqdm(data_iter, total=len(data_iter))
    counter = 0
    for batch in data_iter:
        
        out = model.forward(batch)
        loss = criterion(out, batch.trg_y).item()
        
        total_loss += loss
        data_iter.set_postfix(loss = loss)
        counter +=1
        
    total_loss /= counter
    return total_loss


In [31]:
for epoch in range(num_epochs):
    model.train()
    loss = train_epoch(train_iter, model, criterion).item()
    print('train', loss)
    
    model.eval()
    with torch.no_grad():
        loss = valid_epoch(valid_iter, model, criterion)
        print('valid', loss)

  0%|          | 0/4587 [00:00<?, ?it/s]

train 5.853016376495361


  0%|          | 0/41277 [00:00<?, ?it/s]

valid 5.562058136929399


  0%|          | 0/4587 [00:00<?, ?it/s]

...

valid 5.4533791669916685


  0%|          | 0/4587 [00:00<?, ?it/s]

train 5.571727752685547


100%|██████████| 4587/4587 [04:18<00:00, 17.71it/s, loss=5.91]

valid 5.458160187492761


# Inference

In [32]:
# torch.save(model.state_dict(), "models/model_alldata_bs128.pt")

In [63]:
model.load_state_dict(torch.load(open("models/model_alldata_bs128.pt", "rb"), map_location=torch.device("cpu")))

<All keys matched successfully>

In [64]:
model.to("cpu")
print(f"Moved model to CPU device for inference purpoces")

Moved model to CPU device for inference purpoces


In [78]:
QUESTION = "ты патриот россии?"
ANSWER = "как же без этого?)"

In [66]:
def answer_question(
    question, 
    tokenizer, 
    model, 
    sampling, 
    eos_id=0, 
    max_answer_len=50, 
    max_question_len=50, 
    pad_id=3, 
    bos_id=1
):
    
    tokens = tokenizer.encode(question).ids
    while len(tokens) < max_question_len - 1:
        tokens.append(pad_id)
    tokens = [bos_id] + tokens
    
    input_tokens = torch.tensor(tokens).unsqueeze(0)
    input_mask = (input_tokens != pad_id)
    
    answer_len = 0
    answer_ids = []
    answer_tokens = torch.tensor([bos_id]).unsqueeze(0)
    answer_mask = torch.tensor([True]) 
    
    with torch.no_grad():
        question_encoded = model.encode(input_tokens, src_mask=input_mask) 

        
        while answer_len < max_answer_len:
            pred_decoder = model.decode(
                answer_tokens, 
                answer_mask, 
                question_encoded,
                input_mask
            ) # (batch_size, seq_len, vocab_size)
#             print(f"PredDecoder_SHAPE: {pred_decoder.shape}")
            pred_token_id = sampling(pred_decoder) # returns token id
            pred_token = tokenizer.id_to_token(pred_token_id)
#             print(f"PredToken: {pred_token}, {pred_token_id}")
            
            answer_ids.append(pred_token_id)
            answer_tokens = torch.cat((answer_tokens, 
                                      torch.tensor([pred_token_id]).unsqueeze(0).T.to("cpu")), 
                                      dim=1)
            answer_len += 1
#             print(f"ANSWER_TOKENS (shape {answer_tokens.shape}): {answer_tokens}")
    
    answer = tokenizer.decode(answer_ids)
    return answer

In [67]:
# argmax_sampling

def argmax(tensor_out):
    tokens_probs = torch.softmax(tensor_out, dim=2)
    pred_id = torch.argmax(tokens_probs[0, -1, :]).item()
    return pred_id

answer = answer_question(
    question=QUESTION,
    tokenizer=tokenizer,
    model=model,
    sampling=argmax,
    pad_id=tokenizer.token_to_id("[PAD]"),
    eos_id=tokenizer.token_to_id("[EOS]"),
    bos_id=tokenizer.token_to_id("[BOS]"),
)
print(f"ANSWER: {answer}")

ANSWER: я не не в


In [79]:
# sampling with temperature 

def sampling_with_temperature(tensor_out, temperature=.3, vocab_size=len(TEXT.vocab.stoi)):
    tokens_probs = torch.softmax(tensor_out/temperature, dim=2)
    pred_id = np.random.choice(vocab_size, 1, p=tokens_probs[0,-1,:].cpu().numpy())
    return pred_id[0]

answer = answer_question(
    question=QUESTION,
    tokenizer=tokenizer,
    model=model,
    sampling=sampling_with_temperature,
    pad_id=tokenizer.token_to_id("[PAD]"),
    eos_id=tokenizer.token_to_id("[EOS]"),
    bos_id=tokenizer.token_to_id("[BOS]"),
)
print(f"ANSWER: {answer}")

ANSWER: нет , я не я .


In [69]:
# top k sampling

def top_k_sampling(logits, k=15):

    sorted_idx = torch.argsort(logits, dim=2, descending=True)[0,-1,:][:k]
    sorted_logits = torch.sort(logits, dim=2, descending=True).values[0,-1,:][:k]
    relative_probs = torch.softmax(sorted_logits, dim=0)

    pred_id = np.random.choice(sorted_idx.cpu().numpy(), 1, p=relative_probs.cpu().numpy())
    return pred_id[0]

answer = answer_question(
    question=QUESTION,
    tokenizer=tokenizer,
    model=model,
    sampling=top_k_sampling,
    pad_id=tokenizer.token_to_id("[PAD]"),
    eos_id=tokenizer.token_to_id("[EOS]"),
    bos_id=tokenizer.token_to_id("[BOS]"),
)
print(f"ANSWER: {answer}")

ANSWER: я не не в россии нет . но не в . а в это я ! нет а я не я и не - я


In [80]:
samplings = {
    "argmax" : argmax,
    "temerature": sampling_with_temperature,
    "top-k": top_k_sampling
}

In [81]:
# Comparison of all methods
QUESTION = "пить в новогоднюю ночь чего будем?"

print(f"QUESTION: {QUESTION}")
for sampling_name, sampling_function in samplings.items():
    answer = answer_question(
        question=QUESTION,
        tokenizer=tokenizer,
        model=model,
        sampling=sampling_function,
        pad_id=tokenizer.token_to_id("[PAD]"),
        eos_id=tokenizer.token_to_id("[EOS]"),
        bos_id=tokenizer.token_to_id("[BOS]"),
    )
    print(f"ANSWER ({sampling_name}): {answer}")

QUESTION: пить в новогоднюю ночь чего будем?
ANSWER (argmax): а что ? ? и не только .
ANSWER (temerature): а что с ? и шампанское .
ANSWER (top-k): я уже уже на я буду буду за ! и буду буду ! - ! не пойду - как все с - ) ) прива -


In [36]:
# test data
i = 55

example, answer = test[i].src, test[i].tgt
print(example, answer)

['какие', 'желания', 'женщин', 'не', 'нужно', 'исполнять', '?)'] ['я', 'думаю', '--', 'бредо', 'вые', 'идеи', '.']


# Chat Bot

In [82]:
BOT_TOKEN = "5077708318:AAEIwGbzxwye2t8XzIgjkl0sRs0dvtQL_EI"

In [83]:
import telebot;
bot = telebot.TeleBot(BOT_TOKEN);

In [84]:
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    if message.text == "Привет":
        bot.send_message(message.from_user.id, "Привет, задай мне любой вопрос")
    elif message.text == "/help":
        bot.send_message(message.from_user.id, "Напиши привет")
    else:
        question = message.text
        answer = answer_question(
            question=question,
            tokenizer=tokenizer,
            model=model,
            sampling=sampling_with_temperature,
            pad_id=tokenizer.token_to_id("[PAD]"),
            eos_id=tokenizer.token_to_id("[EOS]"),
            bos_id=tokenizer.token_to_id("[BOS]"),
        )
        
        bot.send_message(message.from_user.id, answer)

In [ ]:
bot.polling(none_stop=True, interval=0)